<a href="https://colab.research.google.com/github/MonikaGost/Winc/blob/master/Module5_Assignment_SharkAttack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1 What are the most dangerous types of sharks to humans?
# 2 Are children more likely to be attacked by sharks?
# 3 Are shark attacks where sharks were provoked more or less dangerous?
# 4 Are certain activities more likely to result in a shark attack?

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv(/attacks.csv)

df

NameError: ignored